In [10]:
import pandas as pd
#selenium, web scraper
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from re import sub, match
from tqdm import tqdm

In [11]:
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.amazon.com/Best-Books-of-the-Year-So-Far/b/ref=bhp_brws_boty19?ie=UTF8&node=3003015011&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=merchandised-search-leftnav&pf_rd_r=T4F5B1C20GNF2222R2HA&pf_rd_r=T4F5B1C20GNF2222R2HA&pf_rd_t=101&pf_rd_p=dfe90ba6-2174-4c8c-9c55-9e9b4d012467&pf_rd_p=dfe90ba6-2174-4c8c-9c55-9e9b4d012467&pf_rd_i=283155')

### Get the books link categories

In [12]:
category_link, category_genre = [], []
#The best books of 2020 so far by category
best_book = driver.find_elements_by_class_name("bxc-grid__container.bxc-grid__container--width-1500")[-1] # 3 elements
#ows of book categories
rows_categories = best_book.find_elements_by_class_name("bxc-grid__row.bxc-grid__row--light ") # 4 elements
#loop for each category row
for idx, val in enumerate(rows_categories):
    #the first itteration is book's banner, so we need to skip that
    if idx == 0:
        continue
    #every row return 4 elements
    category_links = val.find_elements_by_tag_name('a') 
    #loop for each link
    for link in category_links:
        #the url from tag a
        category_link.append(link.get_property('href'))
        #save the book genre to the list
        category_genre.append(link.get_attribute('aria-label'))
sleep(2)

### Go to the link url 

In [39]:
#list of book link, list of general book genre (value from category_genre), list of book status (best seller)
books_link, books_genre, book_status, index = [], [], [], 0

#tqdm is used to display the progress precentage when we run the looping process
for link in tqdm (category_link):
    #go to book link
    driver.get(link)
    #we need stop our progress in 3 second to wait the website loading the data
    sleep(3)
    
    """
    flaq : to check last pg_nation page. If it's the last, the flaq value will assign with false
    page : to check  is it the first page? because the css structure is different between the first and
           the page after we press next pg_nation button. So I need make 2 condition to handle this case.
    """
    flaq, page = True, 0
    #get data for each category with its PG-nation
    while flaq:
        #check the first page
        if page == 0: 
            #container for all books list
            all_book_lists = driver.find_element_by_xpath("/html/body/div[2]/div[3]/div/div[1]/div/div[2]/div[2]")
            #container for each books
            books_data = all_book_lists.find_elements_by_class_name("s-result-item.celwidget") #return list
            #Itteration of the book link
            for data in books_data:
                #title element
                book_title = data.find_element_by_class_name("a-row.a-spacing-small")
                #get a url from the title
                books_link.append(book_title.find_element_by_xpath("div[1]/a").get_property('href'))
                
                #assign books_genre with category_genre, so we will get the same genre before change the book category
                books_genre.append(category_genre[index])
                
                #book_status (best seller, teacher's pick, etc)
                try:
                    #book status container
                    book_reword = data.find_element_by_class_name("a-row.sx-badge-region.sx-pinned-top-badge")
                    #assign the status into array
                    book_status.append(book_reword.find_element_by_xpath("div/a/span[1]/span/span").text)
                except:
                    #we will get a null element if that book doesn't have status container (error handling), assign with ""
                    book_status.append("")
                    continue
        # if this is the second page or more       
        else:
            #container for all books list
            all_book_lists_2 = driver.find_element_by_class_name("s-main-slot.s-result-list.s-search-results.sg-row")
            #container for each books
            books_data =  all_book_lists_2.find_elements_by_class_name("sg-col-20-of-24.s-result-item.s-asin.sg-col-0-of-12.sg-col-28-of-32.sg-col-16-of-20.sg-col.sg-col-32-of-36.sg-col-12-of-16.sg-col-24-of-28")
            #Itteration of the book link
            for data in books_data:
                #title element
                book_title = data.find_element_by_class_name("a-size-mini.a-spacing-none.a-color-base.s-line-clamp-2")
                #get a url from the title
                books_link.append(book_title.find_element_by_xpath("a").get_property('href'))
        
                #books_genre
                books_genre.append(category_genre[index])
                
                #book_status 
                try:
                    book_reword = data.find_element_by_class_name("a-badge-label-inner.a-text-ellipsis")
                    book_status.append(book_reword.find_element_by_xpath("span").text)
                except:
                    book_status.append("")
                    continue
        
        #next button handling 
        try:
            #handling the first page
            page += 1
            #take the url from ">" button
            next_button = driver.find_element_by_class_name("pagnRA").find_element_by_tag_name('a').get_property('href')
            driver.get(next_button)
            sleep(2)
        except:
            try:
                #handling the  second page or more
                next_button = driver.find_element_by_class_name("a-last").find_element_by_tag_name('a').get_property('href')
                driver.get(next_button)
                sleep(2)
            except:
                #when we don't find the next button (mean the itteration already done and go to the next category)
                flaq=False
                break
    index += 1

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [02:43<00:00, 13.59s/it]


In [25]:
buku = pd.DataFrame()
buku['url'] = books_link
buku.shape[0]

272

### scrap the data

In [44]:
month_number = {'january':'01', 'february':'02', 'march':'03', 'april':'04', 'may':'05', 'june':'06', 'july':'07',
               'august':'08', 'september':'09', 'october':'10', 'november':'11', 'december':'12'}
date_number = {'1':'01', '2':'02', '3':'03', '4':'04', '5':'05',
               '6':'06', '7':'07', '8':'08', '9':'09'}

def get_scrape_blueprint(driver, index):
    """
    driver : WebDriver that access the link
    index  : use to access the list of books_genre and book_status
    """
    #the specifict genre on that page
    genre = driver.find_elements_by_class_name('a-link-normal.a-color-tertiary')[2].text
    #the book title
    title = driver.find_element_by_id('productTitle').text
    
    #I used this exception hendling to handle different page structure after clicked the next pg_nation
    author, ratting, total_ratting  = "", 0, 0 
    #author
    try:
        #author value from the first page
        author = driver.find_element_by_class_name("a-size-small.a-link-normal.authorNameLink.a-text-normal").text
    except:
        try:
            #author value on the next page
            container_author = driver.find_elements_by_class_name("author.notFaded")[0]
            author = container_author.find_element_by_xpath("a").text
        except:
            print("author not found")
    #ratting
    try:
        #ratting value from the first page, covert the datatype to float 
        ratting = float(driver.find_element_by_class_name("a-size-base.a-nowrap").find_element_by_class_name("a-size-medium.a-color-base").text.split()[0])
    except:
        #ratting value on the next page, covert the datatype to float
        ratting_container = driver.find_element_by_id("detailBullets_averageCustomerReviews")
        ratting = float(ratting_container.find_element_by_xpath("span[1]/span").get_attribute('title').split()[0])
    #total ratting
    try:
        #the number of total review from the first page, covert the datatype to int
        total_ratting = int(sub(r'[^\d.]', '', driver.find_element_by_class_name('a-row.a-spacing-medium.averageStarRatingNumerical').find_element_by_class_name('a-size-base.a-color-secondary').text.split()[0]))
    except:
        #the number of total review on the next page, covert the datatype to int
        ratting_container = driver.find_element_by_id("detailBullets_averageCustomerReviews")
        review = ratting_container.find_element_by_id("acrCustomerReviewText").text
        total_ratting = int(sub(r'[^\d.]', '', review).split()[0])
    
    #Every book has difference price, a book can have one, 2 or five kinds of price and we need to handle it.
    kindle_price, audiobook_price, hardcover_price, paperback_price, audioplayer_price, board_book_price = 0,0,0,0,0,0
    #element of price list
    container_price_list = driver.find_element_by_class_name("a-unordered-list.a-nostyle.a-button-list.a-horizontal")
    #price list
    get_price_lists = container_price_list.find_elements_by_tag_name('li')
    #price list itteration
    for price_list in get_price_lists:
        try:
            #price type
            kind_price_method = price_list.find_element_by_xpath('span/span[1]/span/a/span[1]').text
            #price value
            price_method = float(price_list.find_element_by_xpath('span/span[1]/span/a/span[2]/span').text.split("$")[1])
            #save the value in the different variable
            if kind_price_method == 'Kindle':
                kindle_price = price_method
            elif kind_price_method == 'Audiobook':
                audiobook_price = price_method
            elif kind_price_method == 'Hardcover':
                hardcover_price = price_method
            elif kind_price_method == 'Paperback':
                paperback_price = price_method
            elif kind_price_method == 'Preloaded Digital Audio Player':
                audioplayer_price = price_method
            elif kind_price_method == 'Board book':
                board_book_price = price_method
        except:
            continue
    
    #product detail
    publication_date, file_size, publisher, word_wise, print_length, language = "", 0, "", "", 0, "" 
    asin, text_to_speach, enhanced_typesetting, x_ray, lending, simultaneous_device_usage, screen_reader = "", "", "", "", "", "", "" 
    #product detail element (ul)
    product_detail_ul = driver.find_elements_by_class_name('a-unordered-list.a-nostyle.a-vertical.a-spacing-none.detail-bullet-list')[0]
    #product detail list (li)
    product_detail_li =product_detail_ul.find_elements_by_tag_name('li')
    #product detail itteration
    for idx, detail in enumerate(product_detail_li): 
        #attribute name
        word = detail.find_element_by_xpath('span/span[1]').text
        if word == 'Word Wise :':
            word_wise = detail.find_element_by_xpath('span/span[2]').text
        elif word == 'Print length :':
            print_length = int(detail.find_element_by_xpath('span/span[2]').text.split()[0]) #convert to int
        #not all booke has publisher date, so I took the publish date from this publisher
        elif word == 'Publisher :':
            #publisher name
            publisher = detail.find_element_by_xpath('span/span[2]').text.split("(")[0].rstrip()
            #publish date
            try:
                #value like : title title title (November 7, 2020)
                publisher_date = detail.find_element_by_xpath('span/span[2]').text.split("(")[1].split(")")[0].split()
                #covert month to number (month_number dictionary)
                month = month_number[publisher_date[0].lower()]
                #convert day to number format (date_number dictionary)
                day = match("\d+", publisher_date[1]).group(0) 
                if day in date_number.keys():
                    day = date_number[match("\d+", publisher_date[1]).group(0)]
                year = publisher_date[2]
                publication_date = year+month+day
            except:
                try:
                    #value like : title title (YBR) title (November 7, 2020)
                    publisher_date = detail.find_element_by_xpath('span/span[2]').text.split("(")[2].split(")")[0].split()
                    #covert month to number (month_number dictionary)
                    month = month_number[publisher_date[0].lower()]
                    #convert day to number format (date_number dictionary)
                    day = match("\d+", publisher_date[1]).group(0) 
                    if day in date_number.keys():
                        day = date_number[match("\d+", publisher_date[1]).group(0)]
                    year = publisher_date[2]
                    publication_date = year+month+day
                except:
                    continue                 
        elif word == 'File size :':
            file_size = int(detail.find_element_by_xpath('span/span[2]').text.split()[0]) #only take the number
        elif word == 'Language: :':
            language = detail.find_element_by_xpath('span/span[2]').text
        elif word == 'ASIN :':
            asin = detail.find_element_by_xpath('span/span[2]').text
        elif word == 'Enhanced typesetting :':
            enhanced_typesetting = detail.find_element_by_xpath('span/span[2]').text
        elif word == 'X-Ray :':
            x_ray = detail.find_element_by_xpath('span/span[2]').text
        elif word == 'Screen Reader :':
            screen_reader = detail.find_element_by_xpath('span/span[2]').text
        elif word == 'Simultaneous device usage :':
            simultaneous_device_usage = detail.find_element_by_xpath('span/span[2]').text
        elif word == 'Text-to-Speech :':
            text_to_speach = detail.find_element_by_xpath('span/span[2]').text
        elif word == 'Lending :':
            lending = detail.find_element_by_xpath('span/span[2]').text
    
    #save data into dictionary
    book_scraper = {
        'general_genre': books_genre[index],
        'book_status': book_status[index],
        'genre': genre,
        'title': title,
        'author': author,
        'ratting': ratting,
        'total_ratting': total_ratting,
        'kindle_price' : kindle_price,
        'audiobook_price' : audiobook_price,
        'hardcover_price' : hardcover_price,
        'paperback_price' : paperback_price,
        'audioplayer_price' : audioplayer_price,
        'board_book_price' : board_book_price,
        'publication_date': publication_date,
        'file_size': file_size,
        'publisher': publisher,
        'word_wise': word_wise,
        'print_length': print_length,
        'language': language,
        'asin': asin,
        'text_to_speach': text_to_speach,
        'enhanced_typesetting': enhanced_typesetting,
        'x_ray': x_ray,
        'lending': lending,
        'simultaneous_device_usage': simultaneous_device_usage,
        'screen_reader': screen_reader,
        'book_link':link
    }
    
    return book_scraper

In [45]:
#save scraper data into book_scrape_result, index to access the list of books_genre and book_status
book_scrape_result, link_number = [], -1

for link in tqdm(books_link):
    #go to the book details page
    driver.get(link)
    sleep(2)
    link_number += 1
    
    #We scrape the data from "Kindle menu" because most of books has this price type, so we need to check first. 
    # whether the kindle price is selected or not
    item_selected = ""
    try: #first page
        #container element of selected price type
        price_list_selected = driver.find_element_by_class_name("a-unordered-list.a-nostyle.a-button-list.a-horizontal")
        #element of selected price type
        selected_price = price_list_selected.find_element_by_class_name("swatchElement.selected")
        #price type
        item_selected = selected_price.find_element_by_xpath("span/span[1]/span/a/span[1]").text
    except:
        try: #next page
            price_list_selected = driver.find_element_by_class_name("a-unordered-list.a-nostyle.a-button-list.a-horizontal")
            selected_price = price_list_selected.find_element_by_class_name("swatchElement.selected.resizedSwatchElement")
            item_selected = selected_price.find_element_by_xpath("span/span[1]/span/a/span[1]").text
        except:
            print("Fail to find kindle button in book-",link_number)
            
    
    if item_selected == 'Kindle':
        try:
            #call get_scrape_blueprint function
            book_scraper = get_scrape_blueprint(driver, link_number)
            #save the risult into array of book_scraper_result
            book_scrape_result.append(book_scraper)
        except:
            print("Fail to crawlling the data in book-",link_number)
    else:
        try:
            #we need to go to kindle page if the selected price type is not kindle
            kindle_button = driver.find_element_by_class_name("a-unordered-list.a-nostyle.a-button-list.a-horizontal")
            #get the kindle url
            kindle_link = kindle_button.find_element_by_xpath('li[1]/span/span[1]/span/a').get_property('href')
            #go to kindle page
            driver.get(kindle_link)
            sleep(3)
            
            #call get_scrape_blueprint function
            book_scraper = get_scrape_blueprint(driver, link_number)
            #save the risult into array of book_scraper_result
            book_scrape_result.append(book_scraper)
        except:
            try:
                #if we just have single list, but it is not kindle. We try to scrape that data if it could
                book_scraper = get_scrape_blueprint(driver, link_number)
                book_scrape_result.append(book_scraper)
            except:
                print("Fail to crawlling book's data in index-",link_number)

 30%|████████████████████████▍                                                        | 82/272 [15:55<32:40, 10.32s/it]

Fail to crawlling book's data in index- 81


100%|████████████████████████████████████████████████████████████████████████████████| 272/272 [50:27<00:00, 11.13s/it]


### Data to csv

In [46]:
#convert the data into dataframe
book_scrape_result = pd.DataFrame(book_scrape_result)
#save the data into csv
book_scrape_result.to_csv("amazone_book.csv")

In [47]:
book_scrape_result.head()

,general_genre,book_status,genre,title,author,ratting,total_ratting,kindle_price,audiobook_price,hardcover_price,...,print_length,language,asin,text_to_speach,enhanced_typesetting,x_ray,lending,simultaneous_device_usage,screen_reader,book_link
0,Biographies and memoirs,,Specific Groups,Untamed,Glennon Doyle,4.7,20658,14.99,0.0,16.72,...,302,English,B07VSZTKJ8,Enabled,Enabled,Enabled,Not Enabled,,,https://www.amazon.com/Untamed-Glennon-Doyle-e...
1,Biographies and memoirs,Best Seller,Biographies & Memoirs,The Splendid and the Vile: A Saga of Churchill...,Erik Larson,4.7,8773,14.99,0.0,18.24,...,546,English,B07TRVW6VX,Enabled,Enabled,Enabled,Not Enabled,,,https://www.amazon.com/Splendid-Vile-Churchill...
2,Biographies and memoirs,Best Seller,Biographies & Memoirs,Hidden Valley Road: Inside the Mind of an Amer...,Robert Kolker,4.6,3751,14.99,0.0,15.99,...,370,English,B07TZYFR71,Enabled,Enabled,Enabled,Not Enabled,,,https://www.amazon.com/Hidden-Valley-Road-Insi...
3,Biographies and memoirs,,Humor & Entertainment,"Wow, No Thank You.: Essays",Samantha Irby,4.3,1168,11.49,0.0,0.00,...,311,English,B07W3J49XM,Enabled,Enabled,Enabled,Not Enabled,,,https://www.amazon.com/Wow-No-Thank-You-Essays...
4,Biographies and memoirs,,Humor & Entertainment,Nobody Will Tell You This But Me: A true (as t...,Bess Kalb,4.4,440,11.99,0.0,12.81,...,208,English,B07TRX3TXP,Enabled,Enabled,Enabled,Not Enabled,,,https://www.amazon.com/Nobody-Will-Tell-You-Th...


### error tracking

In [62]:
#treck the error by access the book url (without looping)
driver.get(buku.iloc[171][0])
sleep(3)
link_number += 13
xx = []

#checking price selected
item_selected = ""
try:
    price_list_selected = driver.find_element_by_class_name("a-unordered-list.a-nostyle.a-button-list.a-horizontal")
    selected_price = price_list_selected.find_element_by_class_name("swatchElement.selected")
    item_selected = selected_price.find_element_by_xpath("span/span[1]/span/a/span[1]").text
except:
    try:
        price_list_selected = driver.find_element_by_class_name("a-unordered-list.a-nostyle.a-button-list.a-horizontal")
        selected_price = price_list_selected.find_element_by_class_name("swatchElement.selected.resizedSwatchElement")
        item_selected = selected_price.find_element_by_xpath("span/span[1]/span/a/span[1]").text
    except:
        print("Fail to find kindle button in book-",link_number)


if item_selected == 'Kindle':
    try:
        book_scraper = get_scrape_blueprint(driver, link_number)
        xx.append(book_scraper)
    except:
        print("Fail to crawlling the data in book-",link_number)
else:
    try:
        #go to kindle page
        kindle_button = driver.find_element_by_class_name("a-unordered-list.a-nostyle.a-button-list.a-horizontal")
        kindle_link = kindle_button.find_element_by_xpath('li[1]/span/span[1]/span/a').get_property('href')
        driver.get(kindle_link)
        sleep(3)

        book_scraper = get_scrape_blueprint(driver, link_number)
        xx.append(book_scraper)
    except:
        book_scraper = get_scrape_blueprint(driver, link_number)
        xx.append(book_scraper)
xx

### Scroll the page

In [ ]:
# SCROLL DOWN (u can use this instruction in every website to scroll down its page)

SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
